# Inception model - Pasquet

Here, we explore the inception model [from Pasquet et al. article.](https://arxiv.org/pdf/1806.06607.pdf):

- First the regular model 
- Then the augmented model with the galacatif redenning
- Finaly, an exploration of probability ditributions as output of the model

### Packages

In [ ]:
import tensorflow as tf

#Checking for GPU access
if tf.test.gpu_device_name() != '/device:GPU:0':
  print('WARNING: GPU device not found.')
else:
  print('SUCCESS: Found GPU: {}'.format(tf.test.gpu_device_name()))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from scipy.stats import median_abs_deviation
from tensorflow.keras.metrics import mse

from tools import *
from model_inception import *

## Data preparation

In [ ]:
# data is stored in the following repo
%ls /global/cfs/cdirs/lsst/groups/PZ/valentin_image_data_temp

In [ ]:
# Let's use here directly the two numpy files actually extracted from 'download'
img_path = '/global/cfs/cdirs/lsst/groups/PZ/valentin_image_data_temp/img_30k.npy'
z_path = '/global/cfs/cdirs/lsst/groups/PZ/valentin_image_data_temp/z_30k.npy'

img = np.load(img_path)
z = np.load(z_path)

In [ ]:
# Preprocessing
scaling = []
for i in range(img.shape[-1]):
    sigma = 1.4826*median_abs_deviation(img[...,i].flatten())
    scaling.append(sigma)

img = np.arcsinh(img / scaling / 3.)

In [ ]:
# Train, validation & test split
data = {}
data['train'] = [img[:15000,...], z[:15000]]
data['val'] = [img[15000:20000,...], z[15000:20000]]
data['test'] = [img[20000:,...], z[20000:]]

In [ ]:
# Check transformed images
plt.imshow(data['train'][0][0, ..., :3]);

## Regular Model training

In [ ]:
model = model_tf2()
model.compile(optimizer='adam', loss=mse)

In [ ]:
#model.summary()

In [ ]:
model.compile(optimizer='adam', loss=mse)

In [ ]:
# Learning rate schedule
LEARNING_RATE=0.001
LEARNING_RATE_EXP_DECAY=0.9
lr_decay = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: LEARNING_RATE * LEARNING_RATE_EXP_DECAY**epoch,
    verbose=True)

# Tensoboard tracking
#tb_callback = tf.keras.callbacks.TensorBoard('./logs/inception', update_freq='batch')

model.compile(optimizer='adam', loss=mse)
history = model.fit(x = data['train'][0], 
          y = data['train'][1],
          batch_size = 64,
          validation_data=(data['val'][0], data['val'][1]),
          steps_per_epoch=len(data['train'][0])//64,
          epochs=25,
          callbacks=[lr_decay])

In [ ]:
history_plot(history, 'Inception Model Loss')

In [ ]:
# Get the prediction
preds = model.predict(data['test'][0]).squeeze()

In [ ]:
# Metrics results
dz, pred_bias, smad, out_frac = metrics(data['test'][1], preds)
print_metrics(pred_bias, smad, out_frac)

In [ ]:
plot_results(data['test'][1], preds, pred_bias, out_frac, smad, 'Inception')

#### Residuals analysis

In [ ]:
# Delta_z histogram
plt.hist(dz, bins=50, color='orange')
plt.title('$ \Delta z$ Distribution', fontsize=18);

In [ ]:
# Delta_z repartition with spectroscopic redshift
plt.scatter(data['test'][1], dz, s=1)
plt.axhline(0.05, color='r', linestyle='--')
plt.axhline(-0.05, color='r', linestyle='--')
plt.xlabel('Spectroscopic Redshift' , fontsize=14)
plt.ylabel('$ \Delta z$', fontsize=16);

## Same model with larger training set 

In [ ]:
!ls /global/cfs/cdirs/lsst/groups/PZ/valentin_image_data_temp

In [ ]:
data_full = np.load('/global/cfs/cdirs/lsst/groups/PZ/valentin_image_data_temp/download')

In [ ]:
z = pd.DataFrame(data_full["labels"][:60000]).z

In [ ]:
img = data_full["cube"][:60000]

In [ ]:
scaling = []
for i in range(img.shape[-1]):
    sigma = 1.4826*median_abs_deviation(img[...,i].flatten())
    scaling.append(sigma)

img = np.arcsinh(img / scaling / 3.)

In [ ]:
large_data = {}
large_data['train'] = [img[:40000,...], z[:40000]]
large_data['val'] = [img[40000:50000,...], z[40000:50000]]
large_data['test'] = [img[50000:,...], z[50000:]]

In [ ]:
large_model = model_tf2()

large_model.compile(optimizer='adam', loss=mse)

In [ ]:
#large_model.summary()

In [ ]:
# Learning rate schedule
LEARNING_RATE=0.001
LEARNING_RATE_EXP_DECAY=0.9
lr_decay = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: LEARNING_RATE * LEARNING_RATE_EXP_DECAY**epoch,
    verbose=True)

# Tensoboard tracking
#tb_callback = tf.keras.callbacks.TensorBoard('./logs/inception', update_freq='batch')


large_history = large_model.fit(x = large_data['train'][0], 
          y = large_data['train'][1],
          batch_size = 64,
          validation_data=(large_data['val'][0], large_data['val'][1]),
          steps_per_epoch=len(large_data['train'][1])//64,
          epochs=25,
          callbacks=[lr_decay])

In [ ]:
history_plot(large_history, 'Large Inception Model Loss')

In [ ]:
# Get the predictions
preds = large_model.predict(large_data['test'][0]).squeeze()

In [ ]:
dz, pred_bias, smad, out_frac = metrics(large_data['test'][1], preds)
print_metrics(pred_bias, smad, out_frac)

In [ ]:
plot_results(large_data['test'][1], preds, pred_bias, out_frac, smad, 'Large Inception')

## Extra feature: Galactic Reddening

One can add extra features and concatenate them after the convolution part of the inception model to improve the estimated redshifts. In Pasquet's article, they propose to use the galactic redenning to do so

In [ ]:
data_download = np.load('/global/cfs/cdirs/lsst/groups/PZ/valentin_image_data_temp/download')

In [ ]:
cat = pd.DataFrame(data_download["labels"][:30000] )
cat.columns

In [ ]:
EBV = cat.EBV
EBV.shape

In [ ]:
plt.hist(EBV, bins=100);

In [ ]:
# Split into train, val and test taking 1/2, 1/4 and 1/4 respectively
EBV_train = EBV[:15000]
EBV_val = EBV[15000:20000]
EBV_test = EBV[20000:]

In [ ]:
model_ebv = model_tf2(with_ebv=True)

In [ ]:
model_ebv.compile(optimizer='adam', loss=mse)

In [ ]:
# Learning rate schedule
LEARNING_RATE=0.001
LEARNING_RATE_EXP_DECAY=0.9
lr_decay = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: LEARNING_RATE * LEARNING_RATE_EXP_DECAY**epoch,
    verbose=True)

# Tensoboard tracking
#tb_callback = tf.keras.callbacks.TensorBoard('./logs/inception_w_EBV', update_freq='batch')

history_ebv = model_ebv.fit(x = [data['train'][0], EBV_train], 
          y = data['train'][1],
          batch_size = 64,
          validation_data=([data['val'][0], EBV_val], data['val'][1]),
          steps_per_epoch=len(data['train'][0])//64,
          epochs=25,
          callbacks=[lr_decay])

In [ ]:
history_plot(history_ebv, 'Inception Model with EBV Loss')

In [ ]:
# Get the predictions
preds_ebv = model_ebv.predict([data['test'][0], EBV_test]).squeeze()

In [ ]:
dz, pred_bias, smad, out_frac = metrics(data['test'][1], preds_ebv)
print_metrics(pred_bias, smad, out_frac)

In [ ]:
plot_results(data['test'][1], preds_ebv, pred_bias, out_frac, smad, 'Inception with extra feature EBV')

In [ ]:
plt.hist2d(data['test'][1], preds_ebv, 64, range=[[0,0.7],[0,0.7]], cmap='gist_stern'); 
plt.gca().set_aspect('equal');
plt.plot([0,0.7],[0,0.7],color='r')
plt.xlabel('Spectroscopic Redshift')
plt.ylabel('Predicted Redshift');

## Probability Distribitions Output 

In [ ]:
model = model_tf2(output_distrib=True)

In [ ]:
negloglik = lambda y, p_y: -p_y.log_prob(y)
model.compile(optimizer='adam', loss=negloglik)

In [ ]:
#model.summary()

In [ ]:
# Learning rate schedule
LEARNING_RATE=0.001
LEARNING_RATE_EXP_DECAY=0.9
lr_decay = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: LEARNING_RATE * LEARNING_RATE_EXP_DECAY**epoch,
    verbose=True)

# Tensoboard tracking
#tb_callback = tf.keras.callbacks.TensorBoard('./logs/inception', update_freq='batch')



history = model.fit(x = data['train'][0], 
              y = data['train'][1],
              batch_size = 64,
              validation_data=(data['val'][0], data['val'][1]),
              steps_per_epoch=len(data['train'][0])//64,
              epochs=10,
              callbacks=[lr_decay])

In [ ]:
history_plot(history, 'Inception Model with distribution output')

In [ ]:
# Beware: differences between model() and model.predict()
# The first gives a distribution and the second an array

yhat = model(np.reshape(data['test'][0][0], (1, 64, 64, 5)))
assert isinstance(yhat, tfd.Distribution)

yhat_ = model.predict(np.reshape(data['test'][0][0], (1, 64, 64, 5)))
assert isinstance(yhat_, np.ndarray)

But what we want here are distributions as outputs and not just arrays. So we need to use the call method from model and the predict method. The problem is that with the call methods, memory issues can arise. For this reason, the predictions need to be performed on batched and not on the full test set at once.

In [ ]:
# List of batch of distributions by calling the model on batches of images
preds_distrib = []
batch_size = 1000
start = 0
length = len(data['test'][0])
while start + batch_size <length:
    preds_distrib.append(model(np.reshape(data['test'][0][start:start+batch_size], (batch_size, 64, 64, 5))))
    start += batch_size
preds_distrib.append(model(np.reshape(data['test'][0][start:], (length - start, 64, 64, 5))))

In [ ]:
preds_distrib

The goal is to be able to have the predicted distributions directly on the whole test set

It is not the case yet, so rest of the code is with 'preds', which is just a sample of the predictions

In [ ]:
# Calculate the prior

import scipy.stats
hist = np.histogram(data['train'][1], 64)
prior = scipy.stats.rv_histogram(hist)

plt.hist(data['train'][1], 100, density=True);
x = np.linspace(0, data['train'][1].max(), 100)
plt.plot(x, prior.pdf(x));

In [ ]:
# This returns the distribution q(z | x) for all clusters
z = np.linspace(0,0.7,100)
logps = []
logps_local = []
for i in range(len(z)):
    for k in range(len(preds_distrib)):
        logps_local.append(preds_distrib[k].log_prob(z[i]).numpy())
    ## Here flatten the list
    flat_list = []
    for sublist in logps_local:
        for item in sublist:
            flat_list.append(item)
    logps.append(flat_list)
    logps_local = []
logps = np.stack(logps)

The posterior can be ploted under the training prior or under a flat prior by dividing the posterior by the prior.

In [ ]:
for i in range(3):
    plt.figure()
    plt.plot(z, np.exp(logps[:,i]), label='posterior under training prior')
    plt.plot(z, np.exp(logps[:,i])/prior.pdf(z), label='posterior under flat prior')
    plt.axvline(data['test'][1][i], color='m', label='True value')
    plt.legend()

To evalute the results of distribution outputs, we can try to extract a point estimate of them and compare with previous results: 
- Mean 
- Mode

#### Mean 

In [ ]:
# With training prior
mean_preds = np.squeeze([preds_distrib[k].mean() for k in range(10)]).flatten()

In [ ]:
dz, pred_bias, smad, out_frac = metrics(data['test'][1], mean_preds)
print_metrics(pred_bias, smad, out_frac)

In [ ]:
plot_results(data['test'][1], mean_preds, pred_bias, out_frac, smad, 'Inception - Mean Point Estimate')

#### Mode 

In [ ]:
# With training prior
mode_preds = z[np.exp(logps).argmax(axis=0)]

In [ ]:
dz, pred_bias, smad, out_frac = metrics(data['test'][1], mode_preds)
print_metrics(pred_bias, smad, out_frac)

In [ ]:
plot_results(data['test'][1], mode_preds, pred_bias, out_frac, smad, 'Inception - Mean Point Estimate')